In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### 1) Write a python program to display all the header tags from wikipedia.org and make data frame.

In [2]:
def wiki_headers(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    header_tags = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    header_text = [tag.text.strip() for tag in header_tags]

    df = pd.DataFrame(header_text, columns=['Header Tags'])
    return df
    
url = 'https://en.wikipedia.org/wiki/Main_Page'
wiki_headers(url)

,Header Tags
0,Main Page
1,Welcome to Wikipedia
2,From today's featured article
3,Did you know ...
4,In the news
5,On this day
6,Today's featured picture
7,Other areas of Wikipedia
8,Wikipedia's sister projects
9,Wikipedia languages


#### 2) Write a python program to display list of respected former presidents of India(i.e. Name , Term of office) from https://presidentofindia.nic.in/former-presidents.htm and make data frame.

In [3]:
def president_details(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    div_tags = soup.find_all("div", class_="presidentListing")

    names = []
    terms = []
    for div in div_tags:
        h3_tag = div.find("h3")
        p_tag = div.find("p")

        name = h3_tag.text.strip().split(' (')[0]
        term = p_tag.text.strip().split(': ')[1]

        names.append(name)
        terms.append(term)

    df = pd.DataFrame({'Name': names, 'Term': terms})
    return df


url = 'https://presidentofindia.nic.in/former-presidents.htm'
president_details(url)

,Name,Term
0,Shri Ram Nath Kovind,"25 July, 2017 to 25 July, 2022"
1,Shri Pranab Mukherjee,"25 July, 2012 to 25 July, 2017"
2,Smt Pratibha Devisingh Patil,"25 July, 2007 to 25 July, 2012"
3,DR. A.P.J. Abdul Kalam,"25 July, 2002 to 25 July, 2007"
4,Shri K. R. Narayanan,"25 July, 1997 to 25 July, 2002"
5,Dr Shankar Dayal Sharma,"25 July, 1992 to 25 July, 1997"
6,Shri R Venkataraman,"25 July, 1987 to 25 July, 1992"
7,Giani Zail Singh,"25 July, 1982 to 25 July, 1987"
8,Shri Neelam Sanjiva Reddy,"25 July, 1977 to 25 July, 1982"
9,Dr. Fakhruddin Ali Ahmed,"24 August, 1974 to 11 February, 1977"


#### 3) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape and make data frame.

a) Top 10 ODI teams in men’s cricket along with the records for matches, points and rating.

In [4]:
def team_rankings(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table')

    headers = []
    for th in table.find_all('th'):
        span = th.find('span', class_="u-hide-mobile")
        if span != None:
            headers.append(span.text.strip())
        else:
            headers.append(th.text.strip())

    rows = []
    for i, tr in enumerate(table.find_all("tr")):
        if i > 9:
            break
        row = []
        for td in tr.find_all("td"):
            span = td.find('span', class_="u-hide-phablet")
            if span != None:
                row.append(span.text.strip())
            else:
                row.append(td.text.strip())
        if row:
            rows.append(row)

    df = pd.DataFrame(rows, columns=headers)
    return df

In [5]:
url = 'https://www.icc-cricket.com/rankings/mens/team-rankings/odi'
team_rankings(url)

,Pos,Team,Matches,Points,Rating
0,1,Australia,23,"2,714",118
1,2,Pakistan,20,"2,316",116
2,3,India,33,"3,807",115
3,4,New Zealand,27,"2,806",104
4,5,England,24,"2,426",101
5,6,South Africa,19,"1,910",101
6,7,Bangladesh,25,"2,451",98
7,8,Afghanistan,10,878,88
8,9,Sri Lanka,21,"1,682",80


b) Top 10 ODI Batsmen along with the records of their team and rating.

In [6]:
def player_rankings(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table')

    headers = []
    for th in table.find_all('th'):
        span = th.find('span', class_="u-hide-mobile")
        if span != None:
            headers.append(span.text.strip())
        else:
            headers.append(th.text.strip())

    pos = []        
    span = table.find('span', class_="rankings-block__pos-number")
    pos.append(span.text.strip())        
    for span in table.find_all('span', class_="rankings-table__pos-number")[:9]:
        pos.append(span.text.strip())

    player = []        
    div = table.find('div', class_="rankings-block__banner--name-large")
    player.append(div.text.strip())        
    for td in table.find_all('td', class_="table-body__cell rankings-table__name name")[:9]:
        player.append(td.text.strip())

    team = []        
    div = table.find('div', class_="rankings-block__banner--nationality")
    team.append(div.text.strip())        
    for span in table.find_all('span', class_="table-body__logo-text")[:9]:
        team.append(span.text.strip())

    rating = []        
    div = table.find('div', class_="rankings-block__banner--rating")
    rating.append(div.text.strip())        
    for td in table.find_all('td', class_="table-body__cell rating")[:9]:
        rating.append(td.text.strip())

    data = {}  
    data[headers[0]] = pos
    data[headers[1]] = player
    data[headers[2]] = team
    data[headers[3]] = rating

    df = pd.DataFrame(data)
    return df

In [7]:
url = 'https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting'
player_rankings(url)

,Pos,Player,Team,Rating
0,1,Babar Azam,PAK,886
1,2,Rassie van der Dussen,SA,777
2,3,Fakhar Zaman,PAK,755
3,4,Imam-ul-Haq,PAK,745
4,5,Shubman Gill,IND,738
5,6,David Warner,AUS,726
6,7,Harry Tector,IRE,722
7,8,Virat Kohli,IND,719
8,9,Quinton de Kock,SA,718
9,10,Rohit Sharma,IND,707


c) Top 10 ODI bowlers along with the records of their team andrating.

In [8]:
url = 'https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling'
player_rankings(url)

,Pos,Player,Team,Rating
0,1,Josh Hazlewood,AUS,705
1,2,Mohammed Siraj,IND,691
2,3,Mitchell Starc,AUS,686
3,4,Matt Henry,NZ,667
4,5,Trent Boult,NZ,660
5,6,Rashid Khan,AFG,659
6,7,Adam Zampa,AUS,652
7,8,Mujeeb Ur Rahman,AFG,637
8,9,Mohammad Nabi,AFG,631
9,10,Shaheen Afridi,PAK,630


#### 4) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape and make data frame.

a) Top 10 ODI teams in women’s cricket along with the records for matches, points and rating.

In [9]:
url = 'https://www.icc-cricket.com/rankings/womens/team-rankings/odi'
team_rankings(url)

,Pos,Team,Matches,Points,Rating
0,1,Australia,21,"3,603",172
1,2,England,28,"3,342",119
2,3,South Africa,26,"3,098",119
3,4,India,27,"2,820",104
4,5,New Zealand,25,"2,553",102
5,6,West Indies,27,"2,535",94
6,7,Thailand,11,821,75
7,8,Bangladesh,14,977,70
8,9,Pakistan,27,"1,678",62


b) Top 10 women’s ODI Batting players along with the records of their team and rating.

In [10]:
url = 'https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting'
player_rankings(url)

,Pos,Player,Team,Rating
0,1,Beth Mooney,AUS,754
1,2,Laura Wolvaardt,SA,732
2,3,Natalie Sciver,ENG,731
3,4,Meg Lanning,AUS,717
4,5,Harmanpreet Kaur,IND,716
5,6,Smriti Mandhana,IND,714
6,7,Chamari Athapaththu,SL,673
7,8,Ellyse Perry,AUS,626
8,9,Tammy Beaumont,ENG,595
9,10,Stafanie Taylor,WI,588


c) Top 10 women’s ODI all-rounder along with the records of their team and rating.

In [11]:
url = 'https://www.icc-cricket.com/rankings/womens/player-rankings/odi/bowling'
player_rankings(url)

,Pos,Player,Team,Rating
0,1,Sophie Ecclestone,ENG,751
1,2,Jess Jonassen,AUS,723
2,3,Shabnim Ismail,SA,722
3,4,Megan Schutt,AUS,704
4,5,Hayley Matthews,WI,660
5,6,Kate Cross,ENG,655
6,7,Ayabonga Khaka,SA,634
7,8,Rajeshwari Gayakwad,IND,617
8,9,Marizanne Kapp,SA,598
9,10,Deepti Sharma,IND,589


#### 5) Write a python program to scrape mentioned news details from https://www.cnbc.com/world/?region=world and make data frame.
i) Headline
ii) Time
iii) News Link

In [12]:
def news_headlines(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    div = soup.find_all('div', class_="RiverPlusCard-cardLeft")

    headline = []
    link = []
    time = []
    for d in div:
        div_h = d.find_all('div', class_="RiverHeadline-headline RiverHeadline-hasThumbnail")
        for h in div_h:
            headline.append(h.text.strip())
            for a in h.find_all('a'):
                l = a.get('href')
                if l.startswith('https'):
                    link.append(l)

        div_t = d.find_all('div', class_="RiverByline-authorBylineContainer")

        for t in set(div_t):
            span = t.find('span', class_="RiverByline-datePublished")
            if span != None:
                time.append(span.text.strip())
            else:
                time.append('N/A')

    df = pd.DataFrame({'Headline': headline, 'Time': time, 'News Link': link})
    return df


url = 'https://www.cnbc.com/world/?region=world'
news_headlines(url)

,Headline,Time,News Link
0,German economy enters recession as first-quart...,5 hours ago,https://www.cnbc.com/2023/05/25/german-economy...
1,Morgan Stanley names 2 chip stocks with 'signi...,4 hours ago,https://www.cnbc.com/2023/05/25/morgan-stanley...
2,Chinese augmented reality glasses maker Nreal ...,4 hours ago,https://www.cnbc.com/2023/05/25/chinese-augmen...
3,Boss of Goldman-backed digital bank Starling t...,4 hours ago,https://www.cnbc.com/2023/05/25/boss-of-goldma...
4,"Call to ‘Buy Japan’ is premature, say Bank of ...",N/A,https://www.cnbc.com/2023/05/25/bofa-expects-w...
5,These 4 stocks could double in price — includi...,N/A,https://www.cnbc.com/2023/05/25/redburn-says-4...
6,UK borrowing rates close in on last year's 'mi...,5 hours ago,https://www.cnbc.com/2023/05/25/uk-borrowing-r...
7,"Canada, Saudi Arabia rekindle diplomatic relat...",6 hours ago,https://www.cnbc.com/2023/05/25/canada-saudi-a...
8,"Debt ceiling talks make progress, but House wi...",N/A,https://www.cnbc.com/2023/05/24/debt-ceiling-n...
9,"Alphabet CEO promises ‘AI pact,’ discussed con...",N/A,https://www.cnbc.com/2023/05/24/alphabet-ceo-p...


#### 6) Write a python program to scrape the details of most downloaded articles from AI in last 90 days.https://www.journals.elsevier.com/artificial-intelligence/most-downloaded-articles Scrape below mentioned details and make data frame.
i) Paper Title
ii) Authors
iii) Published Date
iv) Paper URL

In [13]:
url = 'https://www.journals.elsevier.com/artificial-intelligence/most-downloaded-articles'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

h2 = soup.find_all('h2', class_="sc-1qrq3sd-1 gRGSUS sc-1nmom32-0 sc-1nmom32-1 btcbYu goSKRg")
title = [t.text.strip() for t in h2]

span = soup.find_all('span', class_="sc-1w3fpd7-0 dnCnAO")
authors = [a.text.strip() for a in span]

span = soup.find_all('span', class_="sc-1thf9ly-2 dvggWt")
date = [d.text.strip() for d in span]

a_tag = soup.find_all('a', class_="sc-5smygv-0 fIXTHm")
url = [a.get('href') for a in a_tag]

df = pd.DataFrame({'Paper Title': title,
                   'Authors': authors,
                   'Published Date': date,
                   'Paper URL': url})
df

,Paper Title,Authors,Published Date,Paper URL
0,Reward is enough,"David Silver, Satinder Singh, Doina Precup, Ri...",October 2021,https://www.sciencedirect.com/science/article/...
1,Explanation in artificial intelligence: Insigh...,Tim Miller,February 2019,https://www.sciencedirect.com/science/article/...
2,Creativity and artificial intelligence,Margaret A. Boden,August 1998,https://www.sciencedirect.com/science/article/...
3,Conflict-based search for optimal multi-agent ...,"Guni Sharon, Roni Stern, Ariel Felner, Nathan ...",February 2015,https://www.sciencedirect.com/science/article/...
4,Knowledge graphs as tools for explainable mach...,"Ilaria Tiddi, Stefan Schlobach",January 2022,https://www.sciencedirect.com/science/article/...
5,Law and logic: A review from an argumentation ...,"Henry Prakken, Giovanni Sartor",October 2015,https://www.sciencedirect.com/science/article/...
6,Between MDPs and semi-MDPs: A framework for te...,"Richard S. Sutton, Doina Precup, Satinder Singh",August 1999,https://www.sciencedirect.com/science/article/...
7,Explaining individual predictions when feature...,"Kjersti Aas, Martin Jullum, Anders Løland",September 2021,https://www.sciencedirect.com/science/article/...
8,Multiple object tracking: A literature review,"Wenhan Luo, Junliang Xing and 4 more",April 2021,https://www.sciencedirect.com/science/article/...
9,A survey of inverse reinforcement learning: Ch...,"Saurabh Arora, Prashant Doshi",August 2021,https://www.sciencedirect.com/science/article/...


#### 7) Write a python program to scrape mentioned details from dineout.co.in and make data frame-
i) Restaurant name
ii) Cuisine
iii) Location
iv) Ratings
v) Image URL

The webpage for dineout.co.in is not accessible outside of India. Since, I live in the UK, I was not able to access it for the scraping task.